In [141]:
!pip install -U finance-datareader
!pip install torch torchvision

Requirement already up-to-date: finance-datareader in /usr/local/lib/python3.6/dist-packages (0.9.6)


In [142]:
import pandas as pd
import pandas_datareader as pdr
import FinanceDataReader as fdr
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
import time
from copy import deepcopy # Add Deepcopy for args
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [143]:
%matplotlib inline
#데이터 생성과 화면 표시를 위한 라이브러리 호출
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d


In [144]:
'''import FinanceDataReader as fdr

# 한국거래소 상장종목 전체
df_krx = fdr.StockListing('KRX')
df_krx.head()'''

"import FinanceDataReader as fdr\n\n# 한국거래소 상장종목 전체\ndf_krx = fdr.StockListing('KRX')\ndf_krx.head()"

In [145]:
class StockDataset(Dataset):
    
    def __init__(self, symbol, x_frames, y_frames, start, end):
        
        self.symbol = symbol
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = datetime.datetime(*start)
        self.end = datetime.datetime(*end)
        self.scaler = MinMaxScaler()
        self.data = fdr.DataReader(self.symbol, self.start, self.end)#주식 6가지 특징, 고,시,저,종,거,변
        df_gold = pdr.DataReader('GOLDAMGBD228NLBM', 'fred', start=self.start, end=self.end)#금값
        df_du = pdr.DataReader('POILDUBUSDM', 'fred', start=self.start, end=self.end)#석유
        df_kospi = fdr.DataReader('KS11', self.start, self.end)#코스피
        df_dollar = fdr.DataReader('USD/KRW', self.start, self.end)#원달러환율

        #gold 채우기
        df_gold = df_gold.fillna(value=-1)
        gold = []
        for i in df_gold.values:
            gold.append(i[0])

        check = min(gold)
        for i in range(0,len(gold)):
            if gold[i] == -1:
                gold[i] = check
            else:
                check = gold[i]
        #oil 채우기
        du = df_du.values
        ddu = []
        check = int(list(start)[1])
        c = 0
        for i in self.data.index:
            if i.month == check:
                ddu.append(du[c][0])
            else:
                check += 1
                if check >= 13:
                    check = 1
                c += 1
                if c >= len(du):
                    break
                ddu.append(du[c][0])
        
        self.data['dollar'] = df_dollar[['Open']]
        self.data['kospi'] = df_kospi[['Open']]
        self.data['oil'] = pd.DataFrame(data=ddu, index=self.data.index, columns=['oil'])
        self.data['gold'] = pd.DataFrame(data=gold, index=df_gold.index, columns=['gold'])
        print('len:',len(self.data.index))
        self.data = pd.DataFrame(self.scaler.fit_transform(self.data))
        
    def __len__(self):
        return len(self.data) - (self.x_frames + self.y_frames) + 1
    
    def __getitem__(self, idx):
        idx += self.x_frames
        data = self.data.iloc[idx-self.x_frames:idx+self.y_frames]
        #data = data.apply(lambda x: x//10000)
        data = data.values
        X = data[:self.x_frames]
        y = data[self.x_frames:]
        
        return X, y

In [146]:
class GRU(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size, dropout, use_bn):
        super(GRU, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        self.dropout = dropout
        self.use_bn = use_bn 
        self.norm =nn.BatchNorm1d(self.batch_size)
        self.gru = nn.GRU(self.input_dim, self.hidden_dim*2, self.num_layers)
        self.gru2 = nn.GRU(self.hidden_dim*2, self.hidden_dim, self.num_layers)
        self.regressor = self.make_regressor()

    def make_regressor(self):
        layers = []
        if self.use_bn:
            layers.append(nn.BatchNorm1d(self.hidden_dim))
        layers.append(nn.Dropout(self.dropout))
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim//2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim//2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        x, _ = self.gru(x)
        x, _ = self.gru2(x)
        y_pred = self.regressor(x[-1].view(self.batch_size, -1))
        return y_pred

In [147]:
def metric(y_pred, y_true):
    perc_y_pred = np.exp(y_pred.cpu().detach().numpy())
    perc_y_true = np.exp(y_true.cpu().detach().numpy())
    mae = mean_absolute_error(perc_y_true, perc_y_pred, multioutput='raw_values')
    return mae*100

In [148]:
def train(model, trainset, optimizer, loss_fn, args):
    trainloader = DataLoader(trainset, 
                             batch_size=args.batch_size, 
                             shuffle=True, drop_last=True)
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    train_acc = 0.0
    train_loss = 0.0
    for i, (X, y) in enumerate(trainloader):

        X = X.transpose(0, 1).float().to(args.device)
        y_true = y[:, :, 3].float().to(args.device)
        #print(torch.max(X[:, :, 3]), torch.max(y_true))

        model.zero_grad()
        optimizer.zero_grad()

        y_pred = model(X)
        loss = loss_fn(y_pred.view(-1), y_true.view(-1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += metric(y_pred, y_true)[0]

    train_loss = train_loss / len(trainloader)
    train_acc = train_acc / len(trainloader)
    return model, train_loss, train_acc

In [149]:
def validate(model, valset, loss_fn, args):
    valloader = DataLoader(valset, 
                           batch_size=args.batch_size, 
                           shuffle=False, drop_last=True)
    model.eval()

    val_acc = 0.0
    val_loss = 0.0
    with torch.no_grad():
        for i, (X, y) in enumerate(valloader):

            X = X.transpose(0, 1).float().to(args.device)
            y_true = y[:, :, 3].float().to(args.device)

            y_pred = model(X)
            loss = loss_fn(y_pred.view(-1), y_true.view(-1))

            val_loss += loss.item()
            val_acc += metric(y_pred, y_true)[0]
    val_loss = val_loss / len(valloader)
    val_acc = val_acc / len(valloader)
    return val_loss, val_acc

In [150]:
def experiment(trainset,valset, model, args):

    model.to(args.device)
    loss_fn = torch.nn.MSELoss()

    loss_fn = nn.MSELoss()
    if args.optim == 'SGD':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    # ===================================== #
        
    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        model, train_loss, train_acc = train(model, trainset, optimizer, loss_fn, args)
        val_loss, val_acc = validate(model, valset, loss_fn, args)
        te = time.time()
        
        # ====== Add Epoch Data ====== #
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        # ============================ #
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    return vars(args), result

In [151]:
# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ====== Data Loading ====== #
#삼성전자,Sk하이닉스,네이버,LG화학,셀트리온,현대차,카카오,삼성SDI,LG생활건강,현대모비스
#SK텔레콤,NC소프트,기아차,포스코,KB금융,SK,LG전자,LG,한국전력,삼성생명
args.symbol = ['005930','000660','035420','051910','068270','005380','035720','006400','051900','012330',
               '017670','036570','000270','005490','105560','034730','066570','003550','015760','032830']
args.batch_size = 128
args.x_frames = 10
args.y_frames = 10



In [152]:

# ====== Model Capacity ===== #
args.input_dim = 10
args.hid_dim = 32
args.n_layers = 1

# ====== Regularization ======= #
args.l2 = 0.00001
args.dropout = 0.3
args.use_bn = True

# ====== Optimizer & Training ====== #
args.optim = 'Adam' #'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 0.001
args.epoch = 50


model = GRU(args.input_dim, args.hid_dim, args.y_frames, args.n_layers, args.batch_size, args.dropout, args.use_bn)


print(args)

valset = StockDataset('028050', args.x_frames,args.y_frames,(2011,1,1), (2020,1,1))

for sb in args.symbol:
    trainset = StockDataset(sb, args.x_frames,args.y_frames,(2011,1,1), (2020,1,1))    
    setting, result = experiment(trainset, valset, model,deepcopy(args))

Namespace(batch_size=128, device='cuda', dropout=0.3, epoch=50, hid_dim=16, input_dim=10, l2=1e-05, lr=0.001, n_layers=1, optim='Adam', symbol=['005930', '000660', '035420', '051910', '068270', '005380', '035720', '006400', '051900', '012330', '017670', '036570', '000270', '005490', '105560', '034730', '066570', '003550', '015760', '032830'], use_bn=True, x_frames=10, y_frames=10)
len: 2215
len: 2215
Epoch 0, Acc(train/val): 48.57/30.19, Loss(train/val) 0.32475/0.20470. Took 0.48 sec
Epoch 1, Acc(train/val): 33.55/26.35, Loss(train/val) 0.23585/0.16321. Took 0.46 sec
Epoch 2, Acc(train/val): 24.54/24.21, Loss(train/val) 0.17075/0.13712. Took 0.47 sec
Epoch 3, Acc(train/val): 24.09/25.69, Loss(train/val) 0.12752/0.13018. Took 0.47 sec
Epoch 4, Acc(train/val): 23.74/26.45, Loss(train/val) 0.09767/0.11971. Took 0.48 sec
Epoch 5, Acc(train/val): 22.69/26.73, Loss(train/val) 0.08197/0.10381. Took 0.46 sec
Epoch 6, Acc(train/val): 21.03/28.35, Loss(train/val) 0.07126/0.09814. Took 0.46 sec
E

In [153]:
testinput = StockDataset('028050',args.x_frames,args.y_frames,(2001,1,1),(2020,5,13))
testloader = DataLoader(testinput, batch_size=args.batch_size, shuffle=False, drop_last=True)

print(len(testloader))

len: 4784
37


In [154]:

result = []
result2 = []
for i, (X,y) in enumerate(testloader):

    model.eval()
    X = X.transpose(0, 1).float().to(args.device)
    y_true = y[:, :, 3].float().to(args.device)

    y_pred = model(X)
    y_pred = y_pred.view(-1)
    y_true = y_true.view(-1)
    y_pred = y_pred.cpu().detach().numpy()
    y_true = y_true.cpu().detach().numpy()
    result.extend(y_pred)
    result2.extend(y_true)


In [155]:
print(result)
print(result2)

[0.07588857, 0.07984272, 0.07852338, 0.08191752, 0.080950014, 0.08452018, 0.08368423, 0.084221914, 0.08482233, 0.084544554, 0.07588857, 0.07984272, 0.07852338, 0.08191752, 0.080950014, 0.08452018, 0.08368423, 0.084221914, 0.08482233, 0.084544554, 0.07588857, 0.07984272, 0.07852338, 0.08191752, 0.080950014, 0.08452018, 0.08368423, 0.084221914, 0.08482233, 0.084544554, 0.07588857, 0.07984272, 0.07852338, 0.08191752, 0.080950014, 0.08452018, 0.08368423, 0.084221914, 0.08482233, 0.084544554, 0.07588857, 0.07984272, 0.07852338, 0.08191752, 0.080950014, 0.08452018, 0.08368423, 0.084221914, 0.08482233, 0.084544554, 0.07588857, 0.07984272, 0.07852338, 0.08191752, 0.080950014, 0.08452018, 0.08368423, 0.084221914, 0.08482233, 0.084544554, 0.07588857, 0.07984272, 0.07852338, 0.08191752, 0.080950014, 0.08452018, 0.08368423, 0.084221914, 0.08482233, 0.084544554, 0.07588857, 0.07984272, 0.07852338, 0.08191752, 0.080950014, 0.08452018, 0.08368423, 0.084221914, 0.08482233, 0.084544554, 0.07588857, 0.0

Text(0.5, 1.0, 'pred vs real')

In [156]:
fig = plt.figure(figsize=(40,10))
xl = [n for n in range(1,len(result)+1)]


# ====== test plot ====== #
ax = fig.add_subplot(1, 1, 1)
ax.plot(xl, result,  label='pred')
ax.plot(xl, result2,  label='real')
ax.set_xlabel('len')
ax.set_ylabel('value')
ax.grid()
ax.legend()
ax.set_title('pred vs real')

In [157]:
'''
from google.colab import drive
drive.mount('/content/gdrive')

modelPath = '/content/gdrive/My Drive/GRU_batch_128_model.pt'
torch.save(model.state_dict(), modelPath)
'''

"\nfrom google.colab import drive\ndrive.mount('/content/gdrive')\n\nmodelPath = '/content/gdrive/My Drive/GRU_batch_128_model.pt'\ntorch.save(model.state_dict(), modelPath)\n"